In [16]:
import numpy as np
import pandas as pd
import config_final
import requests
from bs4 import BeautifulSoup as BS


In [17]:
config_final.user

'munzma'

# Webscrape URL List for Bill Summaries

In [58]:
raw = pd.read_csv('bills93-114.csv', sep=";", encoding='latin-1')
raw.dropna(axis=0, subset = ['Cosponsr'], inplace=True)

#raw['URL'] = raw['URL'].astype('str') 
# urls = df[['BillID','URL']]
# df = df.set_index('BillID')

print(raw.shape)

(294485, 48)


In [59]:
len(raw)

294485

## Create Scraper

In [60]:
from bs4 import NavigableString

def scraper(url):
       
    response = requests.get(url)
    soup = BS(response.content, "html.parser")
    soup.prettify() 
    
    bill_summary = soup.find('div', id="bill-summary")
    paragraphs = bill_summary.find_all('p')
    
    if len(paragraphs) > 1:
        return paragraphs[1].get_text(strip=True)
    else:
        node = soup.find('div', id="bill-summary").contents[-1]
#         print(type(node))
        if type(node) == NavigableString:
            return node.strip()
        else:
            print(url) 
            return ''

                 
    

In [61]:
def clean_df(df):
    
    
    df['Class'].fillna(0, inplace=True)

    df['PooleID'].fillna(0, inplace=True)

    df['PLawDate'].fillna(0, inplace=True)

    df['PLawNum'].fillna(0, inplace=True)

    df['Major'].fillna(0, inplace=True)

    df['Minor'].fillna(0, inplace=True)
    
    df = df[df['BillType'].isin(['hr', 's']) ]
    return df

In [62]:
clean_df(raw).head()

,BillID,BillNum,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,MultNo,PLaw,...,PLawDate,PLawNum,ImpBill,SubChRef,SubRankRef,Majority,ReportH,ReportS,Major,Minor
1880,93-HR-1,1,hr,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,3.0,301.0
1881,93-HR-2,2,hr,0,93,1.0,1973-01-03,0.0,1.0,1,...,1974-09-02,Public law 93-406.,1.0,0.0,0.0,1,1,0,5.0,503.0
1882,93-HR-3,3,hr,0,93,2.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,8.0,805.0
1883,93-HR-4,4,hr,0,93,21.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,2.0,208.0
1884,93-HR-5,5,hr,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,6.0,601.0


In [63]:
df = clean_df(raw)

In [64]:
df['BillType'].value_counts()

hr    164698
s      72919
Name: BillType, dtype: int64

In [65]:
df.shape

(237617, 48)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237617 entries, 1880 to 293782
Data columns (total 48 columns):
BillID        237617 non-null object
BillNum       237617 non-null int64
BillType      237617 non-null object
Chamber       237617 non-null int64
Cong          237617 non-null int64
Cosponsr      237617 non-null float64
IntrDate      237617 non-null object
Mult          237617 non-null float64
MultNo        237617 non-null float64
PLaw          237617 non-null int64
Private       237617 non-null int64
Title         237617 non-null object
Veto          237617 non-null int64
Class         237617 non-null float64
ComC          237617 non-null float64
ComR          237617 non-null float64
CumHServ      237617 non-null float64
CumSServ      237617 non-null float64
Delegate      237617 non-null int64
District      237617 non-null float64
DW1           237431 non-null object
FrstConH      237617 non-null float64
FrstConS      237617 non-null float64
Gender        237617 non-null i

## Enter into Database

In [67]:
from sodapy import Socrata
import sqlalchemy as db

In [5]:
def query(q):
    try:
        return pd.read_sql_query(q, engine)
    # Pass when no data is returned    
    except ResourceClosedError:
        pass

def query_list(col, table, distinct = True):
    elts = ['SELECT',
            'DISTINCT' if distinct else '',
            col,
            'FROM',
            table]
    query_str = ' '.join(elts)
    df = query(query_str)
    l = df.iloc[:,0].tolist()
    return l

**Create**

In [6]:
#instantiate the connection and table

engine = db.create_engine(f'mysql+mysqlconnector://{config_final.user}:{config_final.password}@{config_final.host}/con_bills')

connection = engine.connect()
metadata=db.MetaData()

In [7]:
#Create Table 
current_bills_table = db.Table('current_bills', metadata,               
    db.Column('BillID', db.String(40), primary_key=True),
    db.Column('BillType', db.String(20)),
    db.Column('Chamber', db.Boolean()),
    db.Column('Cong', db.Integer()),
    db.Column('Cosponsr', db.Float()),
    db.Column('IntrDate', db.DateTime()),
    db.Column('Mult', db.Boolean()),
    db.Column('PLaw', db.Boolean()),
    db.Column('Title', db.String(500)),
    db.Column('Veto', db.Boolean()),
    db.Column('Class', db.Float()),
    db.Column('District', db.Float()),
    db.Column('FrstConH', db.Float()),
    db.Column('FrstConS', db.Float()),
    db.Column('Gender', db.Boolean()),
    db.Column('MRef', db.Boolean()),
    db.Column('NameFull', db.String(40)),
    db.Column('Party', db.Float()),
    db.Column('Postal', db.String(10)),
    db.Column('PassH', db.Boolean()),
    db.Column('PassS', db.Boolean()),
    db.Column('PLawDate', db.DateTime()),
    db.Column('PLawNum', db.String(40)),
    db.Column('ImpBill', db.Boolean()),
    db.Column('Majority', db.Boolean()),
    db.Column('Major', db.Float()),
    db.Column('Minor', db.Float()),
    db.Column('URL', db.String(100)),
    db.Column('Summary', db.String(2000))
                         
)

In [8]:
metadata.create_all(engine)

In [70]:
df.head()

,BillID,BillNum,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,MultNo,PLaw,...,PLawDate,PLawNum,ImpBill,SubChRef,SubRankRef,Majority,ReportH,ReportS,Major,Minor
1880,93-HR-1,1,hr,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,3.0,301.0
1881,93-HR-2,2,hr,0,93,1.0,1973-01-03,0.0,1.0,1,...,1974-09-02,Public law 93-406.,1.0,0.0,0.0,1,1,0,5.0,503.0
1882,93-HR-3,3,hr,0,93,2.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,8.0,805.0
1883,93-HR-4,4,hr,0,93,21.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,2.0,208.0
1884,93-HR-5,5,hr,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,6.0,601.0


In [80]:
df = df.reindex(index=df.index[::-1])

In [82]:
df.shape

(237617, 48)

In [81]:
df.head()

,BillID,BillNum,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,MultNo,PLaw,...,PLawDate,PLawNum,ImpBill,SubChRef,SubRankRef,Majority,ReportH,ReportS,Major,Minor
293782,114-S-3548,3548,s,1,114,3.0,2016-12-10,0.0,1.0,0,...,0,0,1.0,0.0,0.0,0,0,0,3.0,333.0
293781,114-S-3547,3547,s,1,114,0.0,2016-12-09,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,20.0,2002.0
293780,114-S-3546,3546,s,1,114,0.0,2016-12-09,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,9.0,900.0
293779,114-S-3545,3545,s,1,114,0.0,2016-12-09,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,12.0,1209.0
293778,114-S-3544,3544,s,1,114,0.0,2016-12-09,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,12.0,1209.0


In [83]:
#Test - Convert to dictionary format
final = df.to_dict(orient = 'records')

In [ ]:
for bill in final:
    bill['Summary'] = scraper(bill['URL'])
    query = db.insert(current_bills_table)
    connection.execute(query, bill)

In [ ]:
# query = db.insert(bills)

In [ ]:
# connection.execute(query, final)

In [ ]:
# Stopped at 17778

#use reversed(final) to start from the other side.